## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,76.95,74,96,1.01,overcast clouds
1,1,Jamestown,US,42.0970,-79.2353,73.53,82,100,5.75,overcast clouds
2,2,Mataura,NZ,-46.1927,168.8643,39.47,88,90,2.51,overcast clouds
3,3,Kapaa,US,22.0752,-159.3190,77.32,77,0,16.11,clear sky
4,4,Myitkyina,MM,25.3833,97.4000,79.45,94,68,1.86,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                         (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,76.95,74,96,1.01,overcast clouds
3,3,Kapaa,US,22.0752,-159.3190,77.32,77,0,16.11,clear sky
4,4,Myitkyina,MM,25.3833,97.4000,79.45,94,68,1.86,broken clouds
7,7,Shahreza,IR,32.0339,51.8794,83.53,20,0,18.52,clear sky
8,8,Atuona,PF,-9.8000,-139.0333,77.68,72,21,24.07,few clouds
12,12,Dvinskoy,RU,62.1535,45.1175,75.58,50,84,7.36,broken clouds
17,17,Tautira,PF,-17.7333,-149.1500,78.13,76,25,13.11,light rain
20,20,Butaritari,KI,3.0707,172.7902,82.27,75,100,22.86,light rain
35,35,Ponta Do Sol,PT,32.6667,-17.1000,77.72,62,0,2.28,clear sky
42,42,Bathsheba,BB,13.2167,-59.5167,86.05,79,20,20.71,few clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,76.95,overcast clouds,42.6001,-73.9662,
3,Kapaa,US,77.32,clear sky,22.0752,-159.3190,
4,Myitkyina,MM,79.45,broken clouds,25.3833,97.4000,
7,Shahreza,IR,83.53,clear sky,32.0339,51.8794,
8,Atuona,PF,77.68,few clouds,-9.8000,-139.0333,
12,Dvinskoy,RU,75.58,broken clouds,62.1535,45.1175,
17,Tautira,PF,78.13,light rain,-17.7333,-149.1500,
20,Butaritari,KI,82.27,light rain,3.0707,172.7902,
35,Ponta Do Sol,PT,77.72,clear sky,32.6667,-17.1000,
42,Bathsheba,BB,86.05,few clouds,13.2167,-59.5167,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [37]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   286
Country                286
Max Temp               286
Current Description    286
Lat                    286
Lng                    286
Hotel Name             286
dtype: int64

In [38]:
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286 entries, 0 to 698
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 286 non-null    object 
 1   Country              286 non-null    object 
 2   Max Temp             286 non-null    float64
 3   Current Description  286 non-null    object 
 4   Lat                  286 non-null    float64
 5   Lng                  286 non-null    float64
 6   Hotel Name           286 non-null    object 
dtypes: float64(3), object(4)
memory usage: 17.9+ KB


In [48]:
clean_hotel_df = clean_hotel_df.loc[clean_hotel_df["Hotel Name"] != ""]

In [49]:
# 8a. Create the output File (CSV)
output_data_file = 'Vacation_Search.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>and<dt></dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))